# TG - Data processing - part 2

In [246]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
%matplotlib inline 
import scipy.io as scipy
import os
import math

In [247]:
data_path = "C:/Users/atmiy/OneDrive/Documents/TG/Data/"
csv_path = "C:/Users/atmiy/OneDrive/Documents/TG/Code/"
subject_name = ["sujet2", "sujet4", "sujet6",
                "sujet7", "sujet8", "sujet9", 
                "sujet13", "sujet16", "sujet17", 
                "sujet20", "sujet21"]

level = ["L","M", "H", "R"]

parameters_band = ["HR", "HRV", "Posture", "BR"]

parameters_eye = ["RightEyePupilDiameter",
                  "LeftEyePupilDiameter",
                  "WeightedGazePointX",
                  "WeightedGazePointY"]

block = ["1", "2"]
file_type = ["band", "eye_noprop", "eye_pupil-prop"]

In [248]:
def import_csv_to_dict(csv_path, subject_name, file_type):
    frame = {}
    for sub in subject_name:
        frame[sub] = pd.read_csv(csv_path + sub + "_" + file_type + ".csv")
        frame[sub]["Subject"] = sub
    return frame

In [249]:
def unique_csv(frame, param, subject_name):
    complete = pd.DataFrame(columns=['Time'] + param + ['Block', 'Level', 'Subject'])
    for sub in subject_name:
        complete = pd.concat([complete,frame[sub]])
    return complete

In [250]:
def reduce_eye_to_band(frame, fref, param, level, block, subject):
    final_frame = pd.DataFrame(columns=frame.columns)
    for lvl in level:
        f1 = frame[frame['Level'] == lvl]
        r1 = fref[fref['Level'] == lvl]
        for blk in block:
            
            index1 = 0
            index2 = 0
            
            f2 = f1[f1['Block'] == int(blk)]
            f2.index = range(len(f2))
            r2 = r1[r1['Block'] == int(blk)]
            r2.index = range(len(r2))

            for i in range(len(f2)):

                if index2 < len(r2):
                    if ( f2['Time'][i] >= r2['Time'][index2]) or (i == len(f2) - 1):
                    
                        mean = f2[['Time'] + param][index1:i].mean()
                        mean['Time'] = r2['Time'][index2]
                        mean['Block'] = int(blk)
                        mean['Level'] = lvl
                        mean['Subject'] = subject

                        final_frame = final_frame.append(mean, ignore_index = True)
        
                        index1 = i
                        index2+=1
                
    final_frame = final_frame.sort_values(
            by = "Time", ascending = True)
    final_frame.index = range(len(final_frame))            
    return final_frame

In [258]:
def save_all_csv(frame, subject_name, cap):
    for sub in subject_name:
        frame[sub].to_csv('C:/Users/atmiy/OneDrive/Documents/TG/Code/' + sub + '_' + cap + '.csv', index = False)

In [251]:
band_frames = import_csv_to_dict(csv_path, subject_name, file_type[0])
eye_frames = import_csv_to_dict(csv_path, subject_name, file_type[1])
eye_frames_pupil = import_csv_to_dict(csv_path, subject_name, file_type[2])

In [254]:
eye_frames_reduced = {}
for sub in subject_name:
    eye_frames_reduced[sub] = reduce_eye_to_band(
        eye_frames_pupil[sub], band_frames[sub], parameters_eye, level, block, sub)

In [259]:
save_all_csv(eye_frames_reduced, subject_name, 'eye_reduced')

In [268]:
merge = {}
for sub in subject_name:
    merge[sub] = pd.merge(band_frames[sub], eye_frames_reduced[sub][['Time'] + parameters_eye], on='Time')

In [271]:
save_all_csv(merge, subject_name, 'all')

In [272]:
all_data = pd.DataFrame(columns=merge['sujet2'].columns)
for sub in subject_name:
    all_data = all_data.append(merge[sub], ignore_index = True)

In [279]:
all_data.to_csv('C:/Users/atmiy/OneDrive/Documents/TG/Code/all_data.csv', index = False)